In [1]:
%pylab inline --no-import-all

from ctypes import c_float

# import numpy as np
from astropy.io import fits


from ROOT import gSystem, TFile, TGraphAsymmErrors

from root_numpy import hist2array, tree2array

#from ROOT import VARootIO, VAEffectiveAreaManager, VAEASimpleParameterData


Populating the interactive namespace from numpy and matplotlib


VEGAS libraries (not used for eventDisplay obviously!)

In [2]:
# if gSystem.Load("libVEGASCommon"):
#     print "Problem loading VEGAS Common libraries - please check this before proceeding"
# if gSystem.Load("libVEGASStage6"):
#     print "Problem loading VEGAS Stage 6 libraries - please check this before proceeding"

Load eventdisplay libraries:

In [3]:
if gSystem.Load("$EVNDISPSYS/lib/libVAnaSum.so"):
    print "Problem loading EventDisplay libraries - please check this before proceeding"

# Compatability
Works with VEGAS v2.5.7 or later.

# Data format
the data format is defined at : https://gamma-astro-data-formats.readthedocs.io/en/latest/

# Task List

## Top priorities
* Time Cuts from ROOT file
* Average Azimuth assumes southerly source ....
* Convert to format so can read through a stage 6 runlist file
* Check validity of 30 min runs for EA (compare decent Crab runlist in 30 vs 10 min runs @ different zenith angles).
* Adding ED capability.

## Validation
* Check high stats crab spectra
* Check other spectra
* Check significances/sky maps
* Upper limits

## Documentation
* Save a number of notebooks that show how to do conversion and analysis

## Wish list
* All offset - this will require reworking events list for saving noises etc.
* Event Types - how do spectra compare when produced as "all events" vs. breaking up into 2, 3 and 4 tel events.
* Get window size for noise from root/ea file - at the moment assume it is 7

In [52]:
edFileIO = TFile.Open("/home/thassan/VERITAS/DL3/eventDisplay/54809.anasum.root")

In [53]:
from ROOT import VTimeMask, VEvndispRunParameter, VSkyCoordinatesUtilities
from astropy.time import Time
# runParameters = edFileIO.Get("run_54809/stereo/VAnaSumRunParameter")
runParametersV2 = edFileIO.Get("run_54809/stereo/runparameterV2")
type(runParameters)

ROOT.VEvndispRunParameter

In [54]:
reference_mjd = 53402.0
t_ref = Time(reference_mjd, format='mjd', scale='utc')
seconds_from_reference = (t_ref - Time(start_mjd, format='mjd', scale='utc')).sec
tstart_from_reference = (t_ref - Time(runParametersV2.fDBRunStartTimeSQL, format='mjd', scale='utc')).sec

NameError: name 'start_mjd' is not defined

In [48]:
startDateTime = (runParametersV2.fDBRunStartTimeSQL).split(" ")
stopDateTime = (runParametersV2.fDBRunStoppTimeSQL).split(" ")
print(stopDateTime[0].split("-"))
year, month, day = [int(k) for k in stopDateTime[0].split("-")]
print(int(VSkyCoordinatesUtilities.getMJD( year, month, day)))

['2011', '02', '04']
55596


In [43]:
year, month, day = stopDateTime[0].split("-")
print(year)

2011


This should be the content of "def __fillEVENTS_not_safe__(edFileIO):"

In [118]:
import numpy as np
import logging
from root_numpy import tree2array
from astropy.time import Time

# EventDisplay imports
from ROOT import VEvndispRunParameter, VSkyCoordinatesUtilities, VAnaSumRunParameter

logger = logging.getLogger(__name__)


evt_dict = {}

# FIXME: This should be taken from a common script (by VEGAS also)
reference_mjd = 53402.0

# Load header ,array info and selected event tree ( vegas > v2.5.7)
runSummary = tree2array(edFileIO.Get("total_1/stereo/tRunSummary"))
runNumber = runSummary['runOn'][0]
telConfig = tree2array(edFileIO.Get("run_{}/stereo/telconfig".format(runNumber)))
runParametersV2 = edFileIO.Get("run_{}/stereo/runparameterV2".format(runNumber))
vAnaSumRunParameter = edFileIO.Get("run_{}/stereo/VAnaSumRunParameter".format(runNumber))
selectedEventsTree = tree2array(edFileIO.Get("run_{}/stereo/TreeWithEventsForCtools".format(runNumber)))
# qStatsData = edFileIO.loadTheQStatsData()
# pixelData = edFileIO.loadThePixelStatusData()
# arrayInfo          = edFileIO.loadTheArrayInfo(0)
# cuts = edFileIO.loadTheCutsInfo()

# Get start and stop time within the run.
startDateTime = (runParametersV2.fDBRunStartTimeSQL).split(" ")
stopDateTime = (runParametersV2.fDBRunStoppTimeSQL).split(" ")

start_year, start_month, start_day = [int(k) for k in startDateTime[0].split("-")]
stop_year, stop_month, stop_day = [int(k) for k in stopDateTime[0].split("-")]

start_mjd = VSkyCoordinatesUtilities.getMJD(start_year, start_month, start_day)
stop_mjd = VSkyCoordinatesUtilities.getMJD(stop_year, stop_month, stop_day)

# Number of seconds between reference time and run MJD at 00:00:00:
t_ref = Time(reference_mjd, format='mjd', scale='utc')
seconds_from_reference = (Time(start_mjd, format='mjd', scale='utc') - t_ref).sec
tstart_from_reference = (Time(runParametersV2.fDBRunStartTimeSQL, format='iso', scale='utc') - t_ref).sec
tstop_from_reference = (Time(runParametersV2.fDBRunStoppTimeSQL, format='iso', scale='utc') - t_ref).sec

# Start filling events
avAlt = []
avAz = []
avRA = []
avDec = []

evNumArr = selectedEventsTree['eventNumber']
# This should already have microsecond resolution if stored with double precision.
timeArr = seconds_from_reference + selectedEventsTree['timeOfDay']
raArr = selectedEventsTree['RA']
decArr = selectedEventsTree['DEC']
azArr = selectedEventsTree['Az']
altArr = selectedEventsTree['El']
energyArr = selectedEventsTree['Energy']
# Not used for the moment by science tools.
# nTelArr = selectedEventsTree['NImages']
logger.debug("Start filling events ...")

# for ev in selectedEventsTree:
#     evNumArr.append(selectedEventsTree['eventNumber'])
#     timeArr.append(selectedEventsTree['timeOfDay'])
#     raArr.append(selectedEventsTree['RA'])
#     decArr.append(selectedEventsTree['DEC'])
#     azArr.append(selectedEventsTree['Az'])
#     altArr.append(selectedEventsTree['El'])
#     energyArr.append(selectedEventsTree['Energy'])
#     # nTelArr.append(selectedEventsTree['NImages'])
#
#     avAlt.append(ev.S.fArrayTrackingElevation_Deg)
#     avAz.append(ev.S.fArrayTrackingAzimuth_Deg)
#     avRA.append(ev.S.fArrayTrackingRA_J2000_Rad)
#     avDec.append(ev.S.fArrayTrackingDec_J2000_Rad)

avAlt = np.mean(avAlt)
# Calculate average azimuth angle from average vector on a circle
# https://en.wikipedia.org/wiki/Mean_of_circular_quantities
avAz_deg = np.deg2rad(avAz)
avAz = np.rad2deg(np.arctan2(np.sum(np.sin(avAz_deg)),np.sum(np.cos(avAz_deg))))
avAz = avAz if avAz > 0 else avAz + 360

avRA = np.rad2deg(np.mean(avRA))
avDec = np.rad2deg(np.mean(avDec))
# Filling Event List
evt_dict['EVENT_ID'] = evNumArr
evt_dict['TIME'] = timeArr
evt_dict['RA'] = raArr
evt_dict['DEC'] = decArr
evt_dict['ALT'] = altArr
evt_dict['AZ'] = azArr
evt_dict['ENERGY'] = energyArr
# evt_dict['EVENT_TYPE'] =nTelArr

# Calculate Live Time
# startDateTime = (runParameters.fDBRunStartTimeSQL).split(" ")
# endTime = runHeader.getEndTime()

# startTime_s = float(startTime.getDayNS()) / 1e9
# endTime_s = float(endTime.getDayNS()) / 1e9
# startTime = runHeader.getStartTime()
# endTime = runHeader.getEndTime()

In [124]:
runSummary

array([(54809, 54809, 55596.17204264, 55596.17204264, 83.80028943, 22.02123002, 83.63333155, 22.01447088, 83.63320833, 22.01447222, 83.63320833, 22.01447222, -0., 0., -0.00057786, 0.49988543, 4, 1203., 1203., 80.15031715, -170.66117859, 80.20628753, -170.66117859, 100.840399, 100.840399, 5.43885023, 5.43885023, 173., 27., 4.5      , 0.16666667, 22.87566795, 8.40399002, 0.65742775, 0.2244389 , 0.04319329, 0.08932723, 0.08932723, 15.01571175, 0., 0.),
       (   -1,    -1,     0.        ,     0.        ,  0.        ,  0.        ,  0.        ,  0.        , 83.63320833, 22.01447222, 83.63320833, 22.01447222,  0., 0.,  0.        , 0.        , 0, 1203., 1203., 80.15031715, -170.68042204, 80.20628753, -173.40473938, 100.840399, 100.840399, 5.43885023, 5.43885023, 173., 27., 4.5000002, 0.16666667, 22.87566767, 8.40399001, 0.65742775, 0.22443891, 0.04319329, 0.08932723, 0.08932723, 15.01571182, 0., 0.)],
      dtype=[('runOn', '<i4'), ('runOff', '<i4'), ('MJDOn', '<f8'), ('MJDOff', '<f8'), ('Ta

In [119]:
def produceTelList(telConfig):
    '''Convert the list of telescopes into a string for FITS header
    '''
    telList = ""
    for tel in telConfig['TelID']:
        telList += "T" + str(tel) + ","
    return telList[:-1]

In [122]:
len(telConfig['TelID'])

4

In [121]:
produceTelList(telConfig)

'T0,T1,T2,T3'

In [95]:
vAnaSumRunParameter.fTimeMaskFile

'/lustre/fs19/group/cta/users/thassan/Software/eventDisplay/aux/ParameterFiles/ANASUM.timemask.dat'

In [97]:
vAnaSumRunParameter.fScalarDeadTimeFrac

0.08932723358747655

In [101]:
runSummary['DeadTimeFracOn']

array([0.08932723, 0.08932723])

In [105]:
tstop_from_reference - tstart_from_reference

1203.0

In [104]:
runSummary['tOn']

array([1203., 1203.])

In [110]:
runParametersV2.fTargetRA

83.63333155231746

Reading deadtime from Nathan code

In [71]:
#   sprintf( objName, "run_%d/stereo/vdeadtime", runid ) ;
#   VDeadTime * deadTime = ( VDeadTime* )anasumfile->Get( objName ) ;
#   if ( !deadTime ) 
#   {
#     cout << "  Error, Unable to load VDeadTime object " << objName << " from file " << anasumInputFile << " , exiting..." << endl;
#     return 1 ;
#   }
#   else
#   {
#     cout << "  Loaded VDeadTime object '" << objName << "' ..." << endl;
#   }
#   sprintf( objName, "run_%d/stereo/deadTimeHistograms", runid ) ;
#   TDirectoryFile * deadTimeDir = ( TDirectoryFile* ) anasumfile->Get( objName ) ;
#   deadTime->readHistograms( deadTimeDir ) ;
#   deadTime->calculateDeadTime();
#   deadTime->printDeadTime();

Now in python:

In [78]:
print(runParametersV2.get)

TypeError: count() takes at least 1 argument (0 given)

In [ ]:
deadTime = 

In [70]:
tstop_from_reference - tstart_from_reference

1203.0

In [59]:
runParametersV2.fDBRunStartTimeSQL

'2011-02-04 03:57:42'

In [12]:
timeMask = VTimeMask()
timeMask = f1.Get("run_54809/stereo/timeMask")
type(timeMask)

ROOT.TDirectoryFile

In [13]:
runSummary = tree2array(f1.Get("total_1/stereo/tRunSummary"))

In [14]:
runSummary['MJDOn']

array([55596.17204264,     0.        ])

In [16]:
0.17204264*86400.

14864.484096

1201.6294175000003

In [5]:
eventList = tree2array(f1.Get("run_54809/stereo/TreeWithEventsForCtools"))

In [15]:
eventList['timeOfDay']

array([14263.5729383, 14265.0786412, 14265.6626184, ..., 15462.3345781,
       15464.5196243, 15465.2023558])

In [ ]:
windowSizeForNoise = 7

def decodeConfigMask(mask=15):
    '''Decode the telescope config mask to find the telescpes in the array'''
    tels = []
    if mask >= 8:
        tels.append(4)
        mask -= 8
    if mask >= 4:
        tels.append(3)
        mask -= 4
    if mask >= 2:
        tels.append(2)
        mask -= 2
    if mask >= 1:
        tels.append(1)
        mask -= 1
    return sorted(tels)


def produceTelList(mask):
    '''Convert the list of telescopes into a string for FITS header'''
    telList = ""
    for tel in decodeConfigMask(mask):
        telList += "T" + str(tel) + ","
    return telList[:-1]

# File Generation

## First we need to generate the primary HDU

This contains the information about who wrote the file and the standards that it is written too.
The basic header has some information, we need to complete it to have the following

<pre>
SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                    8 / number of bits per data pixel                  
NAXIS   =                    0 / number of data axes                            
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
TELESCOP= 'VERITAS'            / Telescope                                      
LICENSE = '        '           / Copyright (c) 2018,The VERITAS Collaboration     
</pre>

In [ ]:
i = 3

runs = [54809, 57993, 58456, 59523]
atms = [21, 22, 21, 21]

run = str(runs[i])
atm = str(atms[i])
st5File = "VEGAS/"+run+".med.ED.050.St5_Stereo.root"
eaFile = "VEGAS/EA_na"+atm+"stan_medPoint_050_ED_GRISU.root"
outfile = 'VEGAS/DL3/'+run+'_DL3.fits'

In [ ]:
hdu0 = fits.PrimaryHDU()
hdu0.header.set('TELESCOP', 'VERITAS', 'Telescope')
hdu0.header.set('LICENSE ', '', 'Copyright (c) 2018,The VERITAS Collaboration')
hdu0.header['COMMENT'] = "FITS (Flexible Image Transport System) format is defined in 'Astronomy"
hdu0.header['COMMENT'] = "and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H"

In [ ]:
hdu0.header

## Event Table 
The second hdu is the event table - this includes all of the events that pass the gamma/hadron selection cuts.

To load this we need to read in the VEGAS stage 5 file and select the keys of interest.  Then this can be saved into a table and then the hdu.  

We also need to put a lot of information into the header about the observations.

In [ ]:
vegasFileIO = VARootIO(st5File, True)
runHeader = vegasFileIO.loadTheRunHeader()
selectedEventsTree = vegasFileIO.loadTheCutEventTree()
qStatsData = vegasFileIO.loadTheQStatsData()

arrayInfo = vegasFileIO.loadTheArrayInfo(0)
pixelData = vegasFileIO.loadThePixelStatusData()

In [ ]:
runHeader.printRunHeader()

In [ ]:
selectedEventsTree.Print()

### First we need to generate an array of the data

In [ ]:
evNumArr = []
timeArr = []
raArr = []
decArr = []
azArr = []
altArr = []
energyArr = []
detXArr = []
detYArr = []
nTelArr = []

### Now we can populate the array from the ROOT file

Note: We also need to save some useful information for the average altitude, azimuth, RA and Dec.

I am not sure how best to do the average of the azimuth - I have ignored this for now ...

In [ ]:
avAlt = []
avAz = []
avRA = []
avDec = []
for ev in selectedEventsTree:
    evNumArr.append(ev.S.fArrayEventNum)
    timeArr.append(float(ev.S.fTime.getDayNS())/1e9)
    raArr.append(np.rad2deg(ev.S.fDirectionRA_J2000_Rad))
    decArr.append(np.rad2deg(ev.S.fDirectionDec_J2000_Rad))
    azArr.append(np.rad2deg(ev.S.fDirectionAzimuth_Rad))
    altArr.append(np.rad2deg(ev.S.fDirectionElevation_Rad))
    energyArr.append(ev.S.fEnergy_GeV / 1000.)
    detXArr.append(ev.S.fDirectionXCamPlane_Deg)
    detYArr.append(ev.S.fDirectionYCamPlane_Deg)
    nTelArr.append(ev.S.fImages)
    
    avAlt.append(ev.S.fArrayTrackingElevation_Deg)
    avAz.append(ev.S.fArrayTrackingAzimuth_Deg)
    avRA.append(ev.S.fArrayTrackingRA_J2000_Rad)
    avDec.append(ev.S.fArrayTrackingDec_J2000_Rad)
        
avAlt = np.mean(avAlt)
avAz = np.mean(avAz)
avRA = np.rad2deg(np.mean(avRA))
avDec = np.rad2deg(np.mean(avDec))

### Now we need to generate an HDU

In [ ]:
hdu1 = fits.BinTableHDU.from_columns([
    fits.Column(name='EVENT_ID', format='1J', array=evNumArr), 
    fits.Column(name='TIME', format='1D', array=timeArr, unit="s"), 
    fits.Column(name='RA', format='1E', array=raArr, unit = "deg"), 
    fits.Column(name='DEC', format='1E', array=decArr, unit = "deg"), 
    fits.Column(name='ALT', format='1E', array=altArr, unit = "deg"), 
    fits.Column(name='AZ', format='1E', array=azArr, unit = "deg"), 
    fits.Column(name='ENERGY', format='1E', array=energyArr, unit = "TeV"), 
    fits.Column(name='DETX', format='1E', array=detXArr, unit = "deg"), 
    fits.Column(name='DETY', format='1E', array=detYArr, unit = "deg"),
    fits.Column(name="EVENT_TYPE", format="1J", array=nTelArr)
])
hdu1.name = "EVENTS"

### Header Information

In [ ]:
hdu1.header.set('OBS_ID  ', runHeader.getRunNumber(), 'Run Number')
hdu1.header.set('TELESCOP', 'VERITAS', 'Data from VERITAS')

startTime = runHeader.getStartTime()
endTime = runHeader.getEndTime()

startTime_s = float(startTime.getDayNS()) / 1e9
endTime_s = float(endTime.getDayNS()) / 1e9
hdu1.header.set('DATE-OBS',
                startTime.getString().split()[0],
                'start date (UTC) of obs yy-mm-dd')
hdu1.header.set('TIME-OBS',
                startTime.getString().split()[1],
                'start time (UTC) of obs hh-mm-ss')
hdu1.header.set('DATE-END',
                endTime.getString().split()[0],
                'end date (UTC) of obs yy-mm-dd')
hdu1.header.set('TIME-END',
                endTime.getString().split()[1],
                'end time (UTC) of obs hh-mm-ss')

hdu1.header.set('TSTART  ',
                startTime_s,
                'mission time of start of obs [s]')
hdu1.header.set('TSTOP   ',
                endTime_s,
                'mission time of end of obs [s]')
hdu1.header.set('MJDREFI ',
                int(startTime.getMJDInt()), 'int part of reference MJD [days]')
hdu1.header.set('MJDREFF ', 0., 'fractional part of reference MJD [days]')

hdu1.header.set('TIMEUNIT', 's', 'time unit is seconds since MET start')
hdu1.header.set('TIMESYS ', 'utc', 'time scale is UTC')
hdu1.header.set('TIMEREF ', 'local', 'local time reference')

hdu1.header.set('ONTIME  ', 
                endTime_s - startTime_s,
                'time on target (including deadtime)')
hdu1.header.set('LIVETIME', runHeader.pfRunDetails.fRunNominalLiveTimeSeconds,
                '(dead=ONTIME-LIVETIME) [s] ')
hdu1.header.set('DEADC   ', runHeader.getLiveTimeFrac(),
                'average deadtime fraction [] ')

hdu1.header.set('OBJECT  ', runHeader.getSourceId(), 'observed object')

hdu1.header.set('RA_PNT  ', avRA, 'observation position RA [deg]')
hdu1.header.set('DEC_PNT ', avDec, 'observation position DEC [deg]')
hdu1.header.set('ALT_PNT ', avAlt, 'average altitude of pointing [deg]')
hdu1.header.set('AZ_PNT  ', avAz, 'average azimuth of pointing [deg]')

hdu1.header.set('RA_OBJ  ',
                np.rad2deg(runHeader.getSourceRA()),
                'observation position RA [deg]')
hdu1.header.set('DEC_OBJ ',
                np.rad2deg(runHeader.getSourceDec()),
                'observation position DEC [deg]')

# get the list of telescopes that participate in the event
hdu1.header.set('TELLIST',
                produceTelList(runHeader.fRunInfo.fConfigMask),
                'comma-separated list of tel IDs')
hdu1.header.set('N_TELS', runHeader.pfRunDetails.fTels,
                'number of telescopes in event list')

# other info - weather? pointing mode

hdu1.header.set('EUNIT   ', 'TeV', 'energy unit')
hdu1.header.set('GEOLAT  ', np.rad2deg(arrayInfo.longitudeRad()), 'longitude of array center [deg]')
hdu1.header.set('GEOLON  ', np.rad2deg(arrayInfo.latitudeRad()), 'latitude of array center [deg]')
hdu1.header.set('ALTITUDE', arrayInfo.elevationM(), 'altitude of array center [m]')

# What are these for? - May note be needed, leave out for now.
# hdu1.header.set('DSTYP1', 'TIME    ', 'Data selection type')
# hdu1.header.set('DSUNI1', 's       ', 'Data selection unit')
# hdu1.header.set('DSVAL1', 'TABLE   ', 'Data selection value')
# hdu1.header.set('DSREF1', ':GTI    ', 'Data selection reference')
# hdu1.header.set('DSTYP2', 'POS(RA,DEC)', 'Data selection type')
# hdu1.header.set('DSUNI2', 'deg     ', 'Data selection unit')
# hdu1.header.set('DSVAL2', 'CIRCLE(83.63,22.01,5)', 'Data selection value')
# hdu1.header.set('DSTYP3', 'ENERGY  ', 'Data selection type')
# hdu1.header.set('DSUNI3', 'TeV     ', 'Data selection unit')
# hdu1.header.set('DSVAL3', '0.05:100', 'Data selection value')
# hdu1.header.set('NDSKEYS', '3       ', 'Number of data selections')
# hdu1.header

## Good Time Intervals (GTI)

In [ ]:
goodTimeStart = []
goodTimeStop = []

goodTimeStart.append(startTime_s)
goodTimeStop.append(endTime_s)

In [ ]:
# do we record the unit in this table as well? (it is in seconds)

hdu2 = fits.BinTableHDU.from_columns([
    fits.Column(name='START', format='1D', array=goodTimeStart), 
    fits.Column(name='STOP', format='1D', array=goodTimeStop)
])
hdu2.name = "GTI"
hdu2.header

## Effective Area

Question - what do we need to do about sim vs real spectral index?

### First we need to load the EA file

In [ ]:
effectiveAreaIO = VARootIO(eaFile, True)
effectiveAreaIO.loadTheRootFile()

### Then we need to load the Effective Area

In [ ]:
effectiveAreaManager = VAEffectiveAreaManager()
effectiveAreaManager.loadEffectiveAreas(effectiveAreaIO)
effectiveAreaManager.setUseReconstructedEnergy(False)

### Calculate Average Noise of run

In [ ]:
avNoise = 0
nTels = 0
for telID in decodeConfigMask(runHeader.fRunInfo.fConfigMask):
    avNoise += qStatsData.getCameraAverageTraceVarTimeIndpt(telID-1, windowSizeForNoise, pixelData, arrayInfo)
    nTels += 1

avNoise /= nTels

### Set the EA parameters

In [ ]:
effectiveAreaParameters = VAEASimpleParameterData()
effectiveAreaParameters.fAzimuth = (avAz)
effectiveAreaParameters.fZenith = (90. - avAlt)
effectiveAreaParameters.fNoise = avNoise # this still needs to be worked out
effectiveAreaParameters.fOffset = 0.5 # itterate over this in steps? (0.5deg as for sims?)

# convert to vector of parameters since this is required for a number of steps
effectiveAreaParameters = effectiveAreaManager.getVectorParamsFromSimpleParameterData(effectiveAreaParameters)


### Get the EA

In [ ]:
effectiveArea = effectiveAreaManager.getEffectiveAreaCurve(effectiveAreaParameters)

### Lets Check by Plotting

In [ ]:
x, y, ye = [], [], []
for i in range(effectiveArea.GetN()):
    tmpX, tmpY = ROOT.Double(0), ROOT.Double(0)
    effectiveArea.GetPoint(i, tmpX, tmpY)
    ye.append(effectiveArea.GetErrorY(i))
    effectiveArea.GetErrorX
    x.append(tmpX)
    y.append(tmpY)
        
x = np.array(x)
y = np.array(y)
ye = np.array(ye)

In [ ]:
# this can be removed from the later code but is good for checking
plt.errorbar(x, y, yerr = ye, ls = "", marker = "_")
plt.semilogy()

In [ ]:
energyLow = np.power(10, x - (x[1] - x[0])/2.)
energyHigh = np.power(10, x + (x[1] - x[0])/2.)
thetaLow = [0.0, 1.0]
thetaHigh = [1.0, 2.0]
# ea = np.vstack((y, y))
ea = [y,y]
minEnergy , maxEnergy = c_float(), c_float()
effectiveAreaManager.getSafeEnergyRange(effectiveAreaParameters, 0.5, minEnergy, maxEnergy)

In [ ]:
x = np.array([(energyLow, energyHigh, thetaLow, thetaHigh, ea)], 
             dtype=[('ENERG_LO', '>f4', np.shape(energyLow)), 
                    ('ENERG_HI', '>f4', np.shape(energyHigh)), 
                    ('THETA_LO', '>f4', np.shape(thetaLow)), 
                    ('THETA_HI', '>f4', np.shape(thetaHigh)), 
                    ('EFFAREA', '>f4', np.shape(ea))])

In [ ]:
hdu3 = fits.BinTableHDU(data=x)
hdu3.name = "EFFECTIVE AREA"

hdu3.header.set('TUNIT1 ', 'TeV', "")
hdu3.header.set('TUNIT2 ', 'TeV', "")
hdu3.header.set('TUNIT3 ', 'deg', "")
hdu3.header.set('TUNIT4 ', 'deg', "")
hdu3.header.set('TUNIT5 ', 'm^2', "")

hdu3.header.set('HDUCLASS', 'GADF',
                'FITS file following the GADF data format.')
hdu3.header.set('HDUCLAS1', 'RESPONSE', 'HDU class')
hdu3.header.set('HDUCLAS2', 'EFF_AREA', 'HDU class')
hdu3.header.set('HDUCLAS3', 'POINT-LIKE', 'HDU class')
hdu3.header.set('HDUCLAS4', 'AEFF_2D', 'HDU class')
hdu3.header.set('LO_THRES', minEnergy.value/1000.,
                'Low energy threshold of validity [TeV]')
hdu3.header.set('HI_THRES', maxEnergy.value/1000.,
                'High energy threshold of validity [TeV]')
hdu3.header.set('RAD_MAX ', 0.1, 'Direction cut applied [deg]')

hdu3.columns

In [ ]:
hdu3.header

## Migration Matrix
We want to get pfEnergy_Rec_VS_MC_2D for the correct parameters.

This could be done with a getter within VEGAS - lets try that first!

In [ ]:
a, e = hist2array(effectiveAreaManager.getEnergyBias2D(effectiveAreaParameters), return_edges=True)

In [ ]:
eLow = np.power(10, [e[0][:-1]])[0]
eHigh = np.power(10, [e[0][1:]])[0]

bLow = np.power(10, [e[1][:-1]])[0]
bHigh = np.power(10, [e[1][1:]])[0]

ac = []
for aa in a:
    if np.sum(aa) > 0:
        ab = aa / np.sum(aa*(bHigh - bLow))
    else:
        ab = aa
    try:
        ac = np.vstack((ac, ab))
    except:
        ac = ab
        
ac = ac.transpose()

In [ ]:
# again good for checking - will delete later
plt.imshow(np.log10(ac))
plt.colorbar()

In [ ]:
x = np.array([(eLow, eHigh, bLow, bHigh, [0, 1.0], [1.0, 2.0], [ac, ac])], 
             dtype=[('ETRUE_LO', '>f4', (len(eLow),)), 
                    ('ETRUE_HI', '>f4', (len(eHigh),)), 
                    ('MIGRA_LO', '>f4', (len(bLow),)), 
                    ('MIGRA_HI', '>f4', (len(bLow),)), 
                    ('THETA_LO', '>f4', (2,)), 
                    ('THETA_HI', '>f4', (2,)), 
                    ('MATRIX', '>f4', (2, np.shape(ac)[0], np.shape(ac)[1]))])

hdu4 = fits.BinTableHDU(data=x)
hdu4.name = "ENERGY DISPERSION"

hdu4.header.set('TUNIT1 ', 'TeV', "")
hdu4.header.set('TUNIT2 ', 'TeV', "")
hdu4.header.set('TUNIT5 ', 'deg', "")
hdu4.header.set('TUNIT6 ', 'deg', "")
# hdu3.header.set('TUNIT5 ', 'm^2', "")

hdu4.header.set('HDUCLASS', 'GADF',
                'FITS file following the GADF data format.')
hdu4.header.set('HDUCLAS1', 'RESPONSE', 'HDU class')
hdu4.header.set('HDUCLAS2', 'EDISP', 'HDU class')
hdu4.header.set('HDUCLAS3', 'POINT-LIKE', 'HDU class')
hdu4.header.set('HDUCLAS4', 'EDISP_2D', 'HDU class')
hdu4.header.set('LO_THRES', minEnergy.value/1000.,
                'Low energy threshold of validity [TeV]')
hdu4.header.set('HI_THRES', maxEnergy.value/1000.,
                'High energy threshold of validity [TeV]')
hdu4.header.set('RAD_MAX ', 0.1, 'Direction cut applied [deg]')

hdu4.columns

In [ ]:
hdu4.header

## Write FITS file

In [ ]:
hdulist = fits.HDUList([hdu0, hdu1, hdu2, hdu3, hdu4])
hdulist.writeto(outfile, overwrite=True)